In [73]:
import pydicom
from pydicom.dataset import Dataset, FileMetaDataset
from pydicom.sequence import Sequence
import numpy as np
import os

In [74]:
# Orientation

def file_plane(IOP):
    IOP_round = [round(x) for x in IOP]
    plane = np.cross(IOP_round[0:3], IOP_round[3:6])
    plane = [abs(x) for x in plane]
    if plane[0] == 1:
        return 0    #"Sagittal"
    elif plane[1] == 1:
        return 1    #"Coronal"
    elif plane[2] == 1:
        return 2    #"Transverse"

def newPosition(n,ax,xp_rt,yp_rt,x_rt,y_rt):
    if ax == 0:
        return(xp_rt+n*2*abs(xp_rt)/abs(x_rt))
    else:
        return(yp_rt+n*2*abs(yp_rt)/abs(y_rt))

In [75]:
path = os.getcwd()
path='/Users/Alireza/Desktop/Datasets/'

files = os.listdir(path+'Input')
file_path2=path+'Input/'+files[1]
print(file_path2)

/Users/Alireza/Desktop/Datasets/Input/1.2.826.0.1.3680043.9.3218.1.1.1872364.1022.1591771672055.7043.0-3-104-6m3jni.dcm


In [76]:
dsorg = pydicom.read_file(file_path2, force=True)

IOP = dsorg.ImageOrientationPatient
plane = file_plane(IOP)
planVal=dsorg.ImagePositionPatient[plane]
planVal=float(planVal)
print(plane)
print(dsorg.PatientName)

2
Covid4237


In [77]:
xp_rt=dsorg.ImagePositionPatient[0]
yp_rt=dsorg.ImagePositionPatient[1]

x_rt=dsorg.Columns
y_rt=dsorg.Rows

In [78]:
# Put Contoure pixel cordination Inside file
with open('/Users/Alireza/Desktop/Datasets/borders.npy', 'rb') as f:
    num = np.load(f)

borders=[]
for t in range(len(num)):
    #print(t,num[t])    
    if plane == 0:  #"Sagittal"
        x=planVal
        y=newPosition(num[t][1],0,xp_rt,yp_rt,x_rt,y_rt)
        z=newPosition(num[t][0],1,xp_rt,yp_rt,x_rt,y_rt)
    elif plane == 1:  #"Coronal"
        x=newPosition(num[t][1],0,xp_rt,yp_rt,x_rt,y_rt)
        y=planVal
        z=newPosition(num[t][0],1,xp_rt,yp_rt,x_rt,y_rt)
    elif plane == 2:#  "Transverse"
        x=newPosition(num[t][1],0,xp_rt,yp_rt,x_rt,y_rt)
        y=newPosition(num[t][0],1,xp_rt,yp_rt,x_rt,y_rt)
        z=planVal
    borders.extend([x,y,z])

In [79]:
# borders=[-199,
#  -199,
#  1879.5,
#  199,
#  -199,
#  1879.5,  
#  199,
#  199,
#  1879.5,    
#  -199,
#  199,
#  1879.5, 
#  -199,
#  -199,
#  1879.5,]

In [80]:
#borders

In [81]:
from pydicom.uid import generate_uid
uid1=generate_uid()
uid2=generate_uid()

In [82]:
# File meta info data elements
file_meta = FileMetaDataset()
file_meta.FileMetaInformationGroupLength = 182
file_meta.FileMetaInformationVersion = b'\x00\x01'
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.481.3'
file_meta.MediaStorageSOPInstanceUID = uid1 #'1.2.826.0.1.534147.578.2719282597.2020101685637449'
file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'
file_meta.ImplementationClassUID = '1.2.40.0.13.1.1'
file_meta.ImplementationVersionName = 'dcm4che-2.0'

In [85]:
dsorg.InstanceNumber

"104"

In [69]:
ds = Dataset()

# Main data elements
ds = Dataset()
ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.481.3'
ds.SOPInstanceUID =uid1 #'1.2.826.0.1.534147.578.2719282597.2020101685637449' #uid1
ds.StudyDate =dsorg.StudyDate #'20450916'
ds.StudyTime =dsorg.StudyTime # '000000'
ds.AccessionNumber = ''
ds.Modality = 'RTSTRUCT'
ds.Manufacturer =dsorg.Manufacturer # 'SIEMENS'
ds.ReferringPhysicianName = ''
ds.OperatorsName = ''
ds.ManufacturerModelName = dsorg.ManufacturerModelName # SOMATOM Definition Edge'
ds.PatientName = dsorg.PatientName # 'Covid7175'
ds.PatientID = dsorg.PatientID # 'Covid7175'
ds.PatientBirthDate = dsorg.PatientBirthDate # '19300101'
ds.PatientSex = dsorg.PatientSex # 'F'
ds.SoftwareVersions = dsorg.SoftwareVersions # 'syngo CT VA48A'
ds.StudyInstanceUID = dsorg.StudyInstanceUID #'1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53221.0' # dsOrg.StudyInstanceUID
ds.SeriesInstanceUID = uid2 #'1.2.826.0.1.534147.578.2719282597.2020101685637450' #uid2
ds.StudyID = ''
ds.SeriesNumber = None
ds.FrameOfReferenceUID = dsorg.FrameOfReferenceUID #'1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53224.0' # dsOrg.FrameOfReferenceUID
ds.PositionReferenceIndicator = ''
ds.StructureSetLabel = 'Scaling04'
ds.StructureSetDate ='20201116'
ds.StructureSetTime ='085637'

In [70]:
# Referenced Frame of Reference Sequence
refd_frame_of_ref_sequence = Sequence()
ds.ReferencedFrameOfReferenceSequence = refd_frame_of_ref_sequence

# Referenced Frame of Reference Sequence: Referenced Frame of Reference 1
refd_frame_of_ref1 = Dataset()
refd_frame_of_ref1.FrameOfReferenceUID =dsorg.FrameOfReferenceUID # '1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53224.0' 

# RT Referenced Study Sequence
rt_refd_study_sequence = Sequence()
refd_frame_of_ref1.RTReferencedStudySequence = rt_refd_study_sequence

# RT Referenced Study Sequence: RT Referenced Study 1
rt_refd_study1 = Dataset()
rt_refd_study1.ReferencedSOPClassUID = '1.2.840.10008.3.1.2.3.1'
rt_refd_study1.ReferencedSOPInstanceUID = dsorg.StudyInstanceUID #'1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53221.0' # 

# RT Referenced Series Sequence
rt_refd_series_sequence = Sequence()
rt_refd_study1.RTReferencedSeriesSequence = rt_refd_series_sequence

# RT Referenced Series Sequence: RT Referenced Series 1
rt_refd_series1 = Dataset()
rt_refd_series1.SeriesInstanceUID =dsorg.SeriesInstanceUID #'1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53222.0'

# Contour Image Sequence
contour_image_sequence = Sequence()
rt_refd_series1.ContourImageSequence = contour_image_sequence

# Contour Image Sequence: Contour Image 1
contour_image1 = Dataset()
contour_image1.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
contour_image1.ReferencedSOPInstanceUID = dsorg.SOPInstanceUID #'1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53223.0' 
contour_image1.ReferencedFrameNumber = "1"
contour_image_sequence.append(contour_image1)
rt_refd_series_sequence.append(rt_refd_series1)
rt_refd_study_sequence.append(rt_refd_study1)
refd_frame_of_ref_sequence.append(refd_frame_of_ref1)


# Structure Set ROI Sequence
structure_set_roi_sequence = Sequence()
ds.StructureSetROISequence = structure_set_roi_sequence

# Structure Set ROI Sequence: Structure Set ROI 1
structure_set_roi1 = Dataset()
structure_set_roi1.ROINumber = "1"
structure_set_roi1.ReferencedFrameOfReferenceUID = dsorg.FrameOfReferenceUID #'1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53224.0' # 
structure_set_roi1.ROIName = 'TestScale'
structure_set_roi1.ROIGenerationAlgorithm = ''
structure_set_roi_sequence.append(structure_set_roi1)


# ROI Contour Sequence
roi_contour_sequence = Sequence()
ds.ROIContourSequence = roi_contour_sequence

# ROI Contour Sequence: ROI Contour 1
roi_contour1 = Dataset()

# Contour Sequence
contour_sequence = Sequence()
roi_contour1.ContourSequence = contour_sequence

# Contour Sequence: Contour 1
contour1 = Dataset()

# Contour Image Sequence
contour_image_sequence = Sequence()
contour1.ContourImageSequence = contour_image_sequence

# Contour Image Sequence: Contour Image 1
contour_image1 = Dataset()
contour_image1.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
contour_image1.ReferencedSOPInstanceUID =dsorg.SOPInstanceUID #'1.2.826.0.1.3680043.9.3218.1.1.302475.1985.1592890895061.53223.0' 
contour_image1.ReferencedFrameNumber = "1"
contour_image_sequence.append(contour_image1)

contour1.ContourGeometricType = 'CLOSED_PLANAR'
contour1.NumberOfContourPoints = len(borders)/3#"4"
contour1.ContourNumber = "1"
contour1.ContourData =borders # [-276.91503267973, -162.50000000000, 516.398692810457, 270.222222222222, -162.50000000000, 514.725490196078, 271.895424836601, -162.50000000000, -177.98039215686, -271.89542483660, -162.50000000000, -176.30718954248]
contour_sequence.append(contour1)

roi_contour1.ReferencedROINumber = "1"
roi_contour_sequence.append(roi_contour1)


# RT ROI Observations Sequence
rtroi_observations_sequence = Sequence()
ds.RTROIObservationsSequence = rtroi_observations_sequence

# RT ROI Observations Sequence: RT ROI Observations 1
rtroi_observations1 = Dataset()
rtroi_observations1.ObservationNumber = "1"
rtroi_observations1.ReferencedROINumber = "1"
rtroi_observations1.RTROIInterpretedType = ''
rtroi_observations1.ROIInterpreter = ''
rtroi_observations_sequence.append(rtroi_observations1)


In [71]:
ds.file_meta = file_meta
ds.is_implicit_VR = False
ds.is_little_endian = True


In [72]:
ds.save_as(path+'Output/MyRtTest.dcm', write_like_original=False)

In [48]:
path

'/Users/Alireza/Desktop/Datasets/'